<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_6_part_2_practic/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_text_classification_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

[Посилання](https://www.kaggle.com/datasets/banuprakashv/news-articles-classification-dataset-for-nlp-and-ml/data)

За допомогою `pytorch` проведіть класифікацію тексту:

* Завантажте усі датафрейми
* Добавте до кожного стовпчик `labels` з відповідним йому класом(bussiness, sports, education, etc). назви класів беріть з назв файлів
* Об'єднайте всі датафрейми в один, див [pd.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
* Створіть стовпчик `text` в якому зберігатиметься текст для подальшої роботи. Можети вибрати такі варіанти:
  * Взяти один зі стовпчиків `headline`, `description`, `content`
  * Об'єднати тексти зі стовпчиків `headline`, `description`, `content`
* Видаліть стовпчики `headline`, `description`, `content`, `url`
* Створіть датасет з використанням `Glove`
* Перевірте розмірність даних у датасеті
* Розділіть дані на тренувальні та тестові, створіть `data_loader`
* Створіть модель та натренуйте її
* Виведіть результати класифікації

In [11]:
import pandas as pd
import zipfile
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import GloVe

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
df = pd.read_csv("/content/business_data.csv.zip")

In [14]:
df.head()

,headlines,description,content,url,category
0,Nirmala Sitharaman to equal Morarji Desai’s re...,With the presentation of the interim budget on...,"Sitharaman, the first full-time woman finance ...",https://indianexpress.com/article/business/bud...,business
1,"‘Will densify network, want to be at least no....","'In terms of market share, we aim to double it...",The merger of Tata group’s budget airlines Air...,https://indianexpress.com/article/business/avi...,business
2,Air India group to induct an aircraft every si...,Air India currently has 117 operational aircra...,The Air India group plans to induct one aircra...,https://indianexpress.com/article/business/avi...,business
3,Red Sea woes: Exporters seek increased credit ...,Rising attacks forced shippers to consider the...,Indian exporters have asked the central govern...,https://indianexpress.com/article/business/red...,business
4,Air India group to induct a plane every 6 days...,"Apart from fleet expansion, 2024 will also see...",The Air India group plans to induct one aircra...,https://indianexpress.com/article/business/avi...,business


In [15]:
!pip install torch==2.0.1 torchtext==0.15.2

In [18]:
df = df.drop(columns=['headlines', 'description', 'content', 'url'])

KeyError: "['headlines', 'description', 'content', 'url'] not found in axis"

In [19]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, max_len=100):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        self.label_encoder = LabelEncoder().fit(labels)
        self.vocab = GloVe(name='6B', dim=100)  # Загружаем предобученные вектора GloVe

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        label = self.label_encoder.transform([self.labels.iloc[idx]])[0]
        label = torch.tensor(label, dtype=torch.long)

        text = self.texts.iloc[idx]
        tokens = text.split()

        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        else:
            tokens += [''] * (self.max_len - len(tokens))

        X = self.vocab.get_vecs_by_tokens(tokens, lower_case_backup=True)
        return X, label